# Timing for calculating distance of 10000 distances/frame * 1000 frames

In [1]:
%%file mpi_calc_distance_0_single_traj.py
# (require: mpi4py, numpy)

# always add those lines to your code
import numpy as np
from mpi4py import MPI
from pytraj.parallel import map as pymap
from pytraj import io
import pytraj.common_actions as pyca
from time import time

comm = MPI.COMM_WORLD
# end. you are free to update anything below here

# split remd.x.000 to N cores and do calc_surf in parallel
root_dir = "../../tests/data/nogit/remd/"
traj_name = root_dir + "/remd.x.000"
parm_name = root_dir + "myparm.top"
Nsize = 10**4
indices = np.random.randint(0, 999, size=Nsize*2).reshape(Nsize, 2)

# load to TrajectoryIterator
traj = io.iterload(traj_name, parm_name)

# mapping different chunk of `traj` in N cores
# need to provide `comm`
# save `total_arr` to rank=0
# others: total_arr = None
t0 = time()
total_arr = pymap(comm, pyca.calc_distance, traj, indices, top=traj.top, dtype='ndarray',
                 root=0, n_frames=traj.n_frames/comm.size)
if comm.rank == 0:
    print ("time to calculate 1000 distances/frame * 1000 frames = %2.2f (s) "  % (time() - t0))

if comm.rank != 0:
    assert total_arr is None

if comm.rank == 0:
    # skip final array since its shape might be different from the rest
    t0 = np.asarray(total_arr[:-1]).flatten()
    t1 = np.asarray(total_arr[-1]).flatten()
    t = np.append(t0, t1)
    t_final = t.reshape(traj.n_frames, t.shape[0]/traj.n_frames)
    print ('total array len: ', t_final.shape[0])
    print ('total array len, 1st array: ', t_final[0].shape[0])

    # assert to serial values
    #t2 = pyca.calc_molsurf(traj, "@CA", dtype='ndarray')
    #assert t.shape == t2.shape
    #assert np.any(t == t2) == True


Overwriting mpi_calc_distance_0_single_traj.py


In [2]:
## 1 core

In [3]:
!mpirun -n 1 python mpi_calc_distance_0_single_traj.py

time to calculate 1000 distances/frame * 1000 frames = 4.72 (s) 
('total array len: ', 1000)
('total array len, 1st array: ', 10000)


## 2 cores

In [4]:
!mpirun -n 2 python mpi_calc_distance_0_single_traj.py

time to calculate 1000 distances/frame * 1000 frames = 2.47 (s) 
('total array len: ', 1000)
('total array len, 1st array: ', 10000)


## 4 cores

In [5]:
!mpirun -n 4 python mpi_calc_distance_0_single_traj.py

time to calculate 1000 distances/frame * 1000 frames = 1.36 (s) 
('total array len: ', 1000)
('total array len, 1st array: ', 10000)


## 8 cores

In [6]:
!mpirun -n 8 python mpi_calc_distance_0_single_traj.py

time to calculate 1000 distances/frame * 1000 frames = 0.86 (s) 
('total array len: ', 1000)
('total array len, 1st array: ', 10000)


# Efficiency

In [17]:
print ('2 cores =  %2.2f' % (100 * 4.72/2.47/2))
print ('4 cores =  %2.2f' % (100 * 4.72/1.36/4))
print ('8 cores =  %2.2f' % (100 * 4.72/0.86/8))

2 cores =  95.55
4 cores =  86.76
8 cores =  68.60


# Use in-memory Trajectory

In [5]:
%%file mpi_calc_distance_1_single_traj.py
# (require: mpi4py, numpy)

# always add those lines to your code
import numpy as np
from mpi4py import MPI
from pytraj.parallel import map as pymap
from pytraj import io
import pytraj.common_actions as pyca
from time import time

comm = MPI.COMM_WORLD
# end. you are free to update anything below here

# split remd.x.000 to N cores and do calc_surf in parallel
root_dir = "../../tests/data/nogit/remd/"
traj_name = root_dir + "/remd.x.000"
parm_name = root_dir + "myparm.top"
Nsize = 10**4
indices = np.random.randint(0, 999, size=Nsize*2).reshape(Nsize, 2)

# load to Trajectory
traj = io.iterload(traj_name, parm_name)[:]

# mapping different chunk of `traj` in N cores
# need to provide `comm`
# save `total_arr` to rank=0
# others: total_arr = None
t0 = time()
total_arr = pymap(comm, pyca.calc_distance, traj, indices, top=traj.top, dtype='ndarray',
                 root=0, n_frames=traj.n_frames/comm.size)
if comm.rank == 0:
    print ("time to calculate 1000 distances/frame * 1000 frames = %2.2f (s) "  % (time() - t0))

if comm.rank != 0:
    assert total_arr is None

if comm.rank == 0:
    # skip final array since its shape might be different from the rest
    t0 = np.asarray(total_arr[:-1]).flatten()
    t1 = np.asarray(total_arr[-1]).flatten()
    t = np.append(t0, t1)
    t_final = t.reshape(traj.n_frames, t.shape[0]/traj.n_frames)
    print ('total array len: ', t_final.shape[0])
    print ('total array len, 1st array: ', t_final[0].shape[0])

    # assert to serial values
    #t2 = pyca.calc_molsurf(traj, "@CA", dtype='ndarray')
    #assert t.shape == t2.shape
    #assert np.any(t == t2) == True

Overwriting mpi_calc_distance_1_single_traj.py


In [4]:
!mpirun -n 1 python mpi_calc_distance_1_single_traj.py
!mpirun -n 4 python mpi_calc_distance_1_single_traj.py
!mpirun -n 8 python mpi_calc_distance_1_single_traj.py

time to calculate 1000 distances/frame * 1000 frames = 3.62 (s) 
('total array len: ', 1000)
('total array len, 1st array: ', 10000)
time to calculate 1000 distances/frame * 1000 frames = 0.99 (s) 
('total array len: ', 1000)
('total array len, 1st array: ', 10000)
time to calculate 1000 distances/frame * 1000 frames = 0.66 (s) 
('total array len: ', 1000)
('total array len, 1st array: ', 10000)
